In [1]:
!pip install --upgrade setuptools pip --user

!pip install onnx
!pip install coremltools>=4.1

In [2]:
import sys
import torch
print(f"Python version: {sys.version}, {sys.version_info} ")
print(f"Pytorch version: {torch.__version__} ")

In [3]:
!nvidia-smi

In [4]:
!# Download YOLOv7 code
!git clone https://github.com/WongKinYiu/yolov7
%cd yolov7
!ls

In [5]:
!# Download trained weights
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-tiny.pt

In [6]:
!python detect.py --weights ./yolov7-tiny.pt --conf 0.25 --img-size 640 --source inference/images/horses.jpg

In [7]:
from PIL import Image
Image.open('/content/yolov7/runs/detect/exp/horses.jpg')

In [8]:
# export CoreML model for iOS/MacOS: yolov7-tiny.mlmodel
%cd /content/yolov7/
!python export.py --weights ./yolov7-tiny.pt --img-size 640 640

In [9]:
# show ONNX model
!ls

In [10]:
# Load Coreml-model
import coremltools as ct

model_filename = 'yolov7-tiny.mlmodel'
model = ct.models.MLModel(model_filename)

In [11]:
# Load image
from PIL import Image
import cv2
import numpy as np

im = cv2.imread('/content/yolov7/inference/images/horses.jpg')
im = cv2.resize(im, (640, 640))
print(f" im = {im.shape}")
b = 1
h, w, ch = im.shape

im = Image.fromarray((im).astype('uint8'))
print(f" im = {im}")

In [12]:
def xywh2xyxy(x):
    # Convert nx4 boxes from [x, y, w, h] to [x1, y1, x2, y2] where xy1=top-left, xy2=bottom-right
    y = x.clone() if isinstance(x, torch.Tensor) else np.copy(x)
    y[:, 0] = x[:, 0] - x[:, 2] / 2  # top left x
    y[:, 1] = x[:, 1] - x[:, 3] / 2  # top left y
    y[:, 2] = x[:, 0] + x[:, 2] / 2  # bottom right x
    y[:, 3] = x[:, 1] + x[:, 3] / 2  # bottom right y
    return y

In [ ]:
# Inference only for MacOS and iOS!!!

#y = model.predict({'image': im})  # coordinates are xywh normalized
#if 'confidence' in y:
#  box = xywh2xyxy(y['coordinates'] * [[w, h, w, h]])  # xyxy pixels
#  conf, cls = y['confidence'].max(1), y['confidence'].argmax(1).astype(np.float)
#  y = np.concatenate((box, conf.reshape(-1, 1), cls.reshape(-1, 1)), 1)
#else:
#  k = 'var_' + str(sorted(int(k.replace('var_', '')) for k in y)[-1])  # output key
#  y = y[k]  # output
#
#print(y)